In [1]:
import operator

In [2]:
#### This function checks parantheses but it turns out it was unneccessary and is not used in the rest of the code

def check_parantheses(kb):
    kb_list = kb.split()
    count = 0
    
    for i in kb_list:
        if i == '(':
            count += 1
        elif i == ')':
            count -= 1
        else:
            pass

    if count == 0:
        return kb
    
    else:
        
        while count > 0:
            kb_list.pop(0)
            count -= 1

        while count < 0:
            kb_list.pop(-1)
            count += 1
        s = ''
        for i in kb_list:
            s += i + ' '
        
        return s

In [3]:
###### This code was used during the summer class. It was made in class as an introduction to stacks
 ###I am not claiming to have thought of it, even if it is basic
class Stack:
    def __init__(self):
        self.items = []
        
    def is_empty(self):
        return self.items == []
    
    def push(self, item):
        self.items.append(item)
        
    def pop(self):
        return self.items.pop()
    
    def peek(self):
        return self.items[-1]
    
    def size(self):
        return len(self.items)
    
    def show(self):
        return self.items
    
###### Richard Sarkis, CSC 162, DATA STRUCT&ALGOR IN PYTHON, 08/2019

In [4]:
##### This was also made in the summer class and I am not claiming to have created this code
##### Both the Binary Tree and the Stack classes are necessary for evaluation of the Parsing Tree

class BinaryTree:
    def __init__(self, root_obj):
        self.key = root_obj
        #self.val = val
        
        self.left_child = None
        self.right_child = None
        
    def insert_left(self, new_node):
        if self.left_child is None:
            self.left_child = BinaryTree(new_node)
        else:
            t = BinaryTree(new_node)
            t.left_child = self.left_child
            self.left_child = t
            
    def insert_right(self, new_node):
        if self.right_child is None:
            self.right_child = BinaryTree(new_node)
        else:
            t = BinaryTree(new_node)
            t.right_child = self.right_child
            self.right_child = t
            
    def get_right_child(self):
        return self.right_child
    
    def get_left_child(self):
        return self.left_child
    
    def set_root_val(self, obj):
        self.key = obj
        
    def get_root(self):
        return self.key

    def __repr__(self):
        return f"BinaryTree({repr(self.key)})"
    
    
    #### found this code online solely for the purpose of debugging, not needed otherwise and not claiming it
    ### https://stackoverflow.com/questions/34012886/print-binary-tree-level-by-level-in-python
    def display(self):
        lines, _, _, _ = self._display_aux()
        for line in lines:
            print(line)

    def _display_aux(self):
        """Returns list of strings, width, height, and horizontal coordinate of the root."""
        # No child.
        if self.right_child is None and self.left_child is None:
            line = '%s' % self.key
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        if self.right_child is None:
            lines, n, p, x = self.left_child._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        if self.left_child is None:
            lines, n, p, x = self.right_child._display_aux()
            s = '%s' % self.key
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        left, n, p, x = self.left_child._display_aux()
        right, m, q, y = self.right_child._display_aux()
        s = '%s' % self.key
        u = len(s)
        first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
        second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
        if p < q:
            left += [n * ' '] * (q - p)
        elif q < p:
            right += [m * ' '] * (p - q)
        zipped_lines = zip(left, right)
        lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
        return lines, n + m + u, max(p, q) + 2, n + u // 2

    #### end of online code
    
###### Richard Sarkis, CSC 162, DATA STRUCT&ALGOR IN PYTHON, 08/2019   

In [5]:
##### This code was also created during the Summer class however I adjusted this to work for this situation

def build_parse_tree(fpexp, model): ### creates Parsing Tree 
    fplist = fpexp.split()
    p_stack = Stack()
    e_tree = BinaryTree('')
    p_stack.push(e_tree)
    current_tree = e_tree
    
    for i in fplist:
        #print(i, '\nfirst: ', p_stack.show(),  " current tree: ", current_tree)
        if i == '(':
            #create left child, and descend to it 
            current_tree.insert_left('')
            p_stack.push(current_tree)
            current_tree = current_tree.get_left_child()
                            
        elif i in ['^', 'v', '=>', '<=>']: # added and, or, >, <, <=, >=, == to add to boolean expressions
            #create right child and descend 
            current_tree.set_root_val(i)
            current_tree.insert_right('')
            p_stack.push(current_tree)
            current_tree = current_tree.get_right_child()
            
        elif i == '!':  
            current_tree.set_root_val('fodder') #sets value of root so that it is not traveled to this root when popping stack
            parent = p_stack.pop() # pops item previous item in stack to ensure 'not' will take that spot
            current_tree = parent # ascends to parent
            current_tree.set_root_val(i) # sets 'not' equal to that root 
            p_stack.push(current_tree) # 'not' is added to the stack
            current_tree = current_tree.get_left_child() #moves to next child
            
        elif i == ')':
            current_tree = p_stack.pop()
        else:
            #raise ValueError("invalid expression given!")
            if model != None:
                sub = model[i]
                #if a non-operator is enountered
                if sub == True:
                    current_tree.set_root_val(True) #sets root to True
                    parent = p_stack.pop()
                    current_tree = parent #tree ascend to the parent
                elif sub == False:
                    current_tree.set_root_val(False) #sets root to False
                    parent = p_stack.pop()
                    current_tree = parent #tree ascend to the parent
                else:
                    print('invalid expression was given!')
                    
            else:
                #if a non-operator is enountered
                current_tree.set_root_val(i)
                parent = p_stack.pop()
                current_tree = parent #tree ascend to the parent
        #print('last: ', p_stack.show(), '\n')
    
    
    #e_tree.display()
    return e_tree

###### Richard Sarkis, CSC 162, DATA STRUCT&ALGOR IN PYTHON, 08/2019   

In [6]:
### Also adjusted from the Summer class
def evaluate(parse_tree):
    opers = {'^': operator.and_,
            'v': operator.or_,
    }
    
    left_c = parse_tree.get_left_child()
    right_c = parse_tree.get_right_child()
       
    if left_c and right_c:
        
        if parse_tree.get_root() in opers: #### evaluates the ^ and v operations 
            fn = opers[parse_tree.get_root()]
            return fn(evaluate(left_c), evaluate(right_c))
        
        else:
            if parse_tree.get_root() == '=>': ### evaluates implications 
                return (not evaluate(left_c)) or evaluate(right_c)
                
            elif parse_tree.get_root() == '<=>': ### evaluates biconditionals 
                return (not evaluate(left_c) or evaluate(right_c)) and (not evaluate(right_c) or evaluate(left_c))
  #new code 
    elif left_c and not right_c and parse_tree.get_root() == '!': ### when 'not' operator, it will only have a left child and no right child
        return not evaluate(left_c)
    
    elif left_c and not right_c and parse_tree.get_root() == '':### if the only thing in the tree is an atom it returns the value of the atom
        return evaluate(left_c)
    
    else:
        #base case
        return parse_tree.get_root()
    
###### Richard Sarkis, CSC 162, DATA STRUCT&ALGOR IN PYTHON, 08/2019   

In [7]:
def entails(kb, a):
    
    kb_c = kb
    a_c = a
    
    operations = ['^', '=>', 'v', '!', '<=>']
    atoms = []
    
    kb_list = kb.split() #### splits string into list 
    #table_tree = BinaryTree('Null', None)
        
    for val in kb_list:
        if val not in operations and val != '(' and val != ')' and val not in atoms: ### finds the atoms in the knowledge base 
            atoms.append(val)     
            
    return check_all(kb_c, a, atoms, {})


In [8]:
def check_all(kb, a, atoms, model): ### come back to this 
        
    if atoms == []:  ### ends recursive call if all atoms have a value
        
        pt = build_parse_tree(kb, model) ### evaluates knowledge base given the current model
        ans = evaluate(pt)
        
        #print(model)
        #print(ans)
        
        if ans == True :   ### If the knowledge base is true, we continue to see if the predicate is true
            pt = build_parse_tree(a, model)  ### evaluate predicate in knowldege base 
            ans = evaluate(pt)
            #print(ans)
            return ans
        
        else:
            return True ############# return True when kb is false

    else:
        p = atoms[0]
        r = atoms[1:] 

        model[p] = True ### creates tree starting with the values being true, depth first tree
        first_result = check_all(kb, a, r, model)

        model[p] = False ### moves on to eventually turn the values into False
        second_result = check_all(kb, a, r, model)
        
        if (first_result == True and second_result == False) or (first_result == False and second_result == True):
            return 'Maybe'

        final_result = first_result and second_result ### answer 
        
        return final_result

In [9]:
#p11 = A
#B11 = B
#p12 = C
#P21 = D
#B21 = E
#P22 = F
#P31 = G


#wumpus = '( ( ! P11 ) ^ ( B11 <=> ( P12 v P21 ) ) ^ ( B21 <=> ( ( P11 v P22 ) v P31 ) ) ^ ( ! B11 ) ^ B21 )'
wumpus = '( ( ( ! A ) ^ ( B <=> ( C v D ) ) ) ^ ( ( E <=> ( ( A v F ) v G ) ) ^ ( ( ! B ) ^ E ) ) )'

In [10]:
#y = mythical
#i = immortal
#o = mortal
#a = mammal
#h = horned
#g = magical

unicorn = '( ( ( y => i ) ^ ( ( ! y ) => ( ( ! i ) ^ a ) ) ) ^ ( ( ( i v a ) => h ) ^ ( h => g ) ) )'

In [11]:
#priest:
a = '( A <=> X )'
b = '( B <=> ( Y v Z ) )'
c = '( C <=> ( A ^ B ) )'
d = '( D <=> ( X ^ Y ) )'
e = '( E <=> ( X ^ Z ) )'
f = '( F <=> ( D v E ) )'
#f = '( F <=> ( D ^ E ) )'
g = '( G <=> ( C => F ) )'
h = '( H <=> ( ( G ^ H ) => A ) )'
                                #1           2             c     3               4        5                     6
#knights = '( ( ( ( ( ( ( ( A <=> X ) ^ ( B <=> ( Y v Z ) ) ) ^ ( C <=> ( A ^ B ) ) ) ^ ( E <=> ( X ^ Z ) ) ) ^ ( F <=> ( D v E ) ) ) ^ ( G <=> ( C => F ) ) ) ^ ( H <=> ( ( G ^ I ) => A ) ) ) ) '

knights = '( ( ( ( ( ( ( ' + a + ' ^ ' + b + ' ) ' + '^ ' + c + ' ) ' + '^ ' + d + ' ) ' + '^ ' + e + ' ) ' + '^ ' + f + ' ) ' + '^ ' + g + ' ) ' + '^ ' + h + ' )'


In [12]:
#entails('( P ^ ( P => Q ) )', '( Q )')

#entails(wumpus, '( ! C )') #Maybe True shows there is no pit

#entails(unicorn, '( y )') #True False
#entails(unicorn, '( g )') #True False
#entails(unicorn, '( h )') #True False


#entails(knights, '( ! X )') # True True 
#entails(knights, '( ! Y )') #True True 
#entails(knights, '( ! Z )') # True True 

In [13]:
def get_rid_of_biconditional(Kb): ### gets rid of biconditionals 
    
    kb_list = Kb.split()
    left = []
    right = []
    new_thing = []
    
    limit_left = None
    limit_right = None
    
    if '<=>' not in kb_list: ### ends recursion 
        
        return Kb 
    
    else:
        for i in range(len(kb_list)): ### looks left to right for biconditionals 
            if kb_list[i] == '<=>':

                ###### left ########
                if kb_list[i - 1] == ')':

                    count = 0 ### creates the left side of the biconditional, relies on parantheses to find sides
                    for j in range(i - 1, 0, -1):
                        left.insert(0, kb_list[j])  
                        if kb_list[j] == ')':
                            count += 1
                            
                        elif kb_list[j] == '(':
                            count -= 1
                            if count == 0:
                                if kb_list[j - 1] == '!': ### accounts for negation 
                                    left.insert(0, kb_list[j - 1])
                                    limit_left = j - 1
                                    
                                else:
                                    limit_left = j
                                    
                                break
                else:
                    left.append(kb_list[i -1]) ### if it's only an atom this becomes the left 
                    limit_left = i - 2


                ###### right ######

                if kb_list[i+1] == '(':
                    count = 0 ### creates the right of the biconditional 
                    for j in range(i+1, len(kb_list)):
                        right.append(kb_list[j])
                        if kb_list[j] == '(':
                            count += 1
                        elif kb_list[j] == ')':
                            count -= 1
                            if count == 0:
                                limit_right = j + 1
                                break
                else: 
                    if kb_list[i + 1] == '!': ### account for negative sign
                        if kb_list[i+2] == '(': ### accounts for negation of statement
                            count = 0
                            for j in range(i+2, len(kb_list)):
                                right.append(kb_list[j])
                                if kb_list[j] == '(':
                                    count += 1
                                elif kb_list[j] == ')':
                                    count -= 1
                                    if count == 0:
                                        
                                        limit_right = j + 1
                                        
                                        break
                        else:
                            right.append(kb_list[i + 1]) ### accounts for negation of single atom 
                            right.append(kb_list[i + 2])
                            limit_right = i + 3
                    else:       
                        right.append(kb_list[i + 1]) ####atom
                        limit_right = i + 2

                break
    
    ### creates the new addition of the kb 
    
    new_thing.append('(')
        
    new_thing = new_thing + left
    new_thing.append('=>')
    
    new_thing = new_thing + right
    new_thing.append(')')
        
    new_thing.append('^')
    
    new_thing.append('(')
   
    new_thing = new_thing + right
    new_thing.append('=>')
        
    new_thing = new_thing + left
    
    
    ### adds together the new addition and the rest of the kb that should be unchanged 
    s = ''
    for i in kb_list[:limit_left]:
        s += i + ' '
    for i in new_thing:
        s += i + ' '
    for i in kb_list[limit_right:]:
        s += i + ' '
         
    return get_rid_of_biconditional(s) ### recursively gets rid of the biconditionals (hopefully no biconditionals inside of biconditionals)

In [14]:
def get_rid_of_implications(Kb): ### generally the same as the biconditionals 
    kb_list = Kb.split()
    left = []
    right = []
    new_thing = []
    
    limit_left = None
    limit_right = None
    
    if '=>' not in kb_list: ### ends recursion 
        
        return Kb 
    
    else:
        for i in range(len(kb_list)):
            if kb_list[i] == '=>':

                ###### left ########
                if kb_list[i - 1] == ')': ### creates the left 

                    count = 0
                    for j in range(i - 1, 0, -1): 
                        left.insert(0, kb_list[j]) 
                        if kb_list[j] == ')':
                            count += 1
                        elif kb_list[j] == '(':
                            count -= 1
                            if count == 0:
                                limit_left = j
                                break
                        #else:
                            #left.insert(0, kb_list[j]) 
                    left.insert(0, '!') ### negates the left of the implication 
                else:
                    
                    if kb_list[i - 2] == '!': ### accounts for negation 
                        left.append('(')
                        left.append('!')
                        left.append('!')
                        left.append(kb_list[i - 1])
                        limit_left = i - 3
                        
                    else:
                        left.append('(') ### normal atom 
                        left.append('!')
                        left.append(kb_list[i - 1]) 
                        limit_left = i - 2
                        
                ###### right ######
                if kb_list[i+1] == '(': ### creates right 
                    count = 0
                    for j in range(i+1, len(kb_list)):
                        right.append(kb_list[j])
                        if kb_list[j] == '(':
                            count += 1
                        elif kb_list[j] == ')':
                            count -= 1
                            if count == 0:
                                limit_right = j + 1
                                break
                else:
                    
                    if kb_list[i + 1] == '!': ### accounts for negation 
                        if kb_list[i+1] == '(':
                            count = 0
                            for j in range(i+1, len(kb_list)):
                                right.append(kb_list[j])
                                if kb_list[j] == '(':
                                    count += 1
                                elif kb_list[j] == ')':
                                    count -= 1
                                    if count == 0:
                                        limit_right = j + 1
                                        break
                        else:
                            right.append(kb_list[i + 1])
                            right.append(kb_list[i + 2])
                            limit_right = i + 3
                            
                    
                    else:
                        
                        right.append(kb_list[i + 1]) ### normal atom 
                        limit_right = i + 2
                
                break       
    
    ### creates the new addition of kb 
    new_thing = left + ['v'] + right    
        
    s = ''
    for i in kb_list[:limit_left]:
        s += i + ' '
    for i in new_thing:
        s += i + ' '
    for i in kb_list[limit_right:]:
        s += i + ' '
        
    return get_rid_of_implications(s) ### recursively check if there are implictaions 

In [15]:
def distribute_negation(Kb):
    kb_list = Kb.split()
    new_thing = []
    sub = []
    
    limit_left = None
    limit_right = None
    
    operations = ['v', '^', '!']
    
        
    for i in range(len(kb_list)):
 
        if kb_list[i] == '!' and kb_list[i + 1] == '!': ## if there are two negations in a sequence then gets rid of them
            kb_list.pop(i)
            kb_list.pop(i)
                        
            s = ''
            for j in kb_list: 
                s = s + j + ' '
            
            return distribute_negation(s) ### because we are popping the negations we have to return the kb in a new occurance        
            
        elif kb_list[i] == '!' and kb_list[i + 1] == '(': ### distribution scenario 
            
            limit_left = i
                        
            sub.append('(')

            count = 0
 
            for j in range(i + 2, len(kb_list)): ### looks for everything to distribute the negation to 

                ### multiple scenarios 
                
                if kb_list[j] == '(': 
                    sub.append('!')
                    sub.append('(')
                    count += 1

                elif kb_list[j] == ')':
                    sub.append(')')
                    count -= 1
                    if count == -1:
                        limit_right = j + 1
                        break 

                elif count == 0 and (kb_list[j] == 'v' or kb_list[j] == '^'): ### changes the sign 
                    
                    if kb_list[j] == 'v':
                        sub.append('^')
                        
                    if kb_list[j] == '^':
                        sub.append('v')
                                    
                else:
                    if count == 0 and kb_list[j] != '!': ### if atom 
                        
                        sub.append('!')
                        sub.append(kb_list[j])
                        
                    else:
                        
                        sub.append(kb_list[j]) ### add everything else 

                    
            
            break
    
    
    if sub == []: ### if we went through and there are no cases of distribution end recursion
        return Kb

    new_thing = kb_list[:limit_left] + sub +  kb_list[limit_right:] ### creates new addition 

    s = ''
    for i in new_thing:
        s += i + ' '
   #print(s, '\n')
    
    return distribute_negation(s) ### recursively looks for more cases

In [16]:
def distribute(Kb): ### distributes nested operations
    
    this_kb = distribute_left(Kb) ## start with left 
    
    return distribute_right(this_kb) ## end with right

In [17]:
def distribute_left(Kb): ### finds cases in which we need to distribute from the left
    
    kb_list = Kb.split()
    
    left = []
    right = []
    new_thing = []
    sub = []
    limit_left = None
    limit_right = None
    
    ult_count = 0
    
    for i in range(len(kb_list) - 1, 0, -1): ### find all the nested left distributions by starting from the right
        
        if kb_list[i] == '(':
            ult_count -= 1
            
        elif kb_list[i] == ')':
            ult_count += 1
        
        ### this is when we find the thing that needs to be distributed 
        if (kb_list[i] == 'v' or kb_list[i] == '^') and ult_count >= 1 and kb_list[i - 1] != ')' and kb_list[i + 1] == '(':
            
            
            ###### left ######
            if kb_list[i - 2] == '!': ### account for negation of left side 
                
                left = [kb_list[i - 2], kb_list[i - 1], kb_list[i]]
                limit_left = i - 3
                
            else: ### normal atom 
                
                left = [kb_list[i - 1], kb_list[i]]
                limit_left = i - 2
            
            ##### right #####
            count = ult_count
            for j in range(i + 1, len(kb_list)):
                
                right.append(kb_list[j])
                
                if kb_list[j] == '(': #### relies on parantheses to find right side 
                    
                    count += 1     

                elif kb_list[j] == ')':
                    
                    count -= 1
                    if count == ult_count:
                        limit_right = j
                        break

            break
            
    if left == []: #ends recursion 
        return Kb
    
    
    s = ''
    for i in right:
        s = s + i + ' '
    
    sub_right = []
    for i in list(distribute_right(s)):  ### if there are right distributions inside of this case it will distribute the inside
        if i != ' ':
            sub_right.append(i)
    
    right = sub_right
            
    bad_stuff = ['v', '^', '!']
    
    ### distributes 
    
    count = 0
    for i in range(len(right)):
        
        if right[i] == '(':
            sub.append('(')
            if count == 1: ### if we found something we need to distribute to then we add it to the right part 
                for i in left:
                    sub.append(i)
                sub.append('(')
            count += 1
            
        elif right[i] == ')':
            sub.append(')')
            count -= 1

        elif (right[i] == 'v' or right[i] == '^'):
            if count == 1: ### adds the parentheses when necessary 
                sub.append(')')
                
            sub.append(right[i])
     
        else:
            
            if count == 1 and right[i] not in bad_stuff and right[i - 1] != '!': ### if normal atom distribute to it
                sub.append('(')
                for j in left:
                    sub.append(j)
                    
                sub.append(right[i])
                
            elif count == 1 and right[i] == '!':
                sub.append('(')
                
                for j in left:
                    sub.append(j)
                    
                sub.append('!')
            
            else:
                sub.append(right[i])
                
    new_thing = kb_list[:limit_left] + sub + kb_list[limit_right + 1:] ### creates new kb 
    
    
    s = ''
    for i in new_thing:
        s = s + i + ' '
    
    return distribute_left(s) ### recursively looks for distribution cases 
    

In [18]:
def distribute_right(Kb): ### after we're done distributing the left we distribute the right
    
    #this is also used in the distribute left function to find nested distribution cases 
    kb_list = Kb.split()
    
    left = []
    right = []
    new_thing = []
    sub = []
    limit_left = None
    limit_right = None
    
    ult_count = 0
    
    for i in range(len(kb_list)): ### essentially the same as left 
                
        if kb_list[i] == '(':
            ult_count += 1
            
        elif kb_list[i] == ')':
            ult_count -= 1
            
        ### if we found a case we need to distribute to 
        if (kb_list[i] == 'v' or kb_list[i] == '^') and ult_count >= 1 and kb_list[i - 1] == ')' and kb_list[i + 1] != '(':

            if kb_list[i + 1] == '!': ### accounts for negation 
                
                right = [kb_list[i], kb_list[i + 1], kb_list[i + 2]]
                limit_right = i + 3
                
            else: ### normal atom 
                
                right = [kb_list[i], kb_list[i + 1]]
                limit_right = i + 2
                
            count = ult_count 
            for j in range(i - 1, 0, -1): ### creates the left 
                
                left.insert(0, kb_list[j])
                
                if kb_list[j] == ')':
                    
                    count += 1
                
                elif kb_list[j] == '(':
                    
                    count -= 1
                    if count == ult_count:
                        limit_left = j
                        break
                        
            break
            
            
    if right == []: ### if there were no cases found then we end recursion 
        return Kb
    
    bad_stuff = ['v', '^', '!']
    
    count = 0
    #sub.append('(')
    for i in range(len(left)):
        
        if left[i] == '(':
            sub.append('(')
            count += 1
            
        elif left[i] == ')':
            sub.append(')')
            count -= 1
            if count == 1: ### if we found something we need to distribute to then we distribute to it
                for j in right:
                    sub.append(j)
                sub.append(')')

        elif (left[i] == 'v' or left[i] == '^'):
            
            sub.append(left[i])
            if count == 1:
                sub.append('(') ### adds parantheses when necessary 
                
        else:
            
            if count == 1 and left[i] not in bad_stuff: ### if we found an atom to distribute to we distribute to it 
                sub.append(left[i])
                for j in right:
                    sub.append(j)
                sub.append(')')
            
            else:
                sub.append(left[i])
                
            
    #print(kb_list[:limit_left], kb_list[limit_right:])  
    new_thing = kb_list[:limit_left] + sub + kb_list[limit_right + 1:]
    
    s = ''
    for i in new_thing:
        s = s + i + ' '
        
    return distribute_right(s) ### recursively looking for these cases
                    
                    
                    
                    

In [19]:
def convert_CNF(Kb): ### converts to CNF 
    #print(Kb, '\n')
    this_kb = get_rid_of_biconditional(Kb)
    #print(this_kb, '\n')
    this_kb = get_rid_of_implications(this_kb)
    #print(this_kb, '\n')
    this_kb = distribute_negation(this_kb)
    #print(this_kb, '\n')
    this_kb = distribute(this_kb)
    #print(this_kb)
    
    return this_kb

In [20]:
### finds the clauses of the kb 
def find_clauses(kb):
    kb_list = kb.split()
    clause_list = []
    
    first_mark = None
    second_mark = None
    
    count = 0
    for i in range(len(kb_list)): ### looks for ^  and next ^
        
        if kb_list[i] == '(':
            
            if count == 1:
                first_mark = i ### makes first mark when appropriate
            count += 1
            
        elif kb_list[i] == ')':
            
            count -= 1
            if count == 1:
                second_mark = i + 1 ### makes second mark in the appropriate case 
            
                
        elif count == 1 and kb_list[i] == '^' and kb_list[i + 1] != '(': ### if the next item is an atom or negated atom
            
            first_mark = i + 1
            for j in range(i + 1, len(kb_list)):
                if (kb_list[j] == '^' and count == 1) or j == len(kb_list):
                    second_mark = j
                    break
                    
        elif count == 1 and kb_list[i] == '^' and kb_list[i - 1] != ')': ### if the previous item is an item 
            
            second_mark = i
            for j in range(i - 1, 0, -1):
                if (kb_list[j] == '^' and count == 1):

                    first_mark = j + 1
                    break
                
        if second_mark != None:
            
            if kb_list[first_mark:second_mark] not in clause_list: ### the clause is the between these two ^ signs 
                clause_list.append(kb_list[first_mark:second_mark]) ### adds clause 
                
            first_mark = None
            second_mark = None

    for i in clause_list:
        if i[-1] != ')':
            i.append(')')
        if i[0] != '(':
            i.insert(0, '(')
    
    if clause_list == []:
        clause_list.append(kb_list)
        
    return clause_list

In [21]:
def clause_final(l): ### this will continually look for nested conjunctions, l is the list of clauses
        
    mark = None
    
    sub = []
    
    x = None
    y = None
    
    for i in range(len(l)):
        count = 0
        for j in range(len(l[i])): ### for each item in the clause
            if l[i][j] == '(':
                count += 1
                
            elif l[i][j] == ')':
                count -= 1
                
            elif l[i][j] == '^' and count == 1:
                mark = j
                x = l[i][:mark] ### creates the left of the nested conjunction 
                y = l[i][mark + 1:] ### creates the right of the nested conjunction 
                if x[-1] != ')':
                    x.append(')')
                if y[0] != '(':
                    y.insert(0, '(')
                
                
                if x[0] == '(' and x[1] == '(': ### don't need but don't want to touch it either
                    x.pop(0)
                if y[-1] == ')' and y [-2] == ')':
                    y.pop(-1)
                break 

        if mark != None: ### gets rid of the old clause
            l.pop(i)
            break
            
    if x == None:
        return l
           
    if x not in l:
        l.append(x) ### adds new clause 
        
    if y not in l:    
        l.append(y)
        
    return clause_final(l)
            

In [22]:
def simplify(a, b): ### simplifies two lists/clauses

    ret_list = []
    
    change = 'no'

    for i in range(len(a)):
        
        if a[i] == '!':
            pass
        
        elif a[i - 1] != '!': ### checks for items to cancel starting with atoms that are not negated 
            
            for j in range(len(b)):
                if b[j] == a[i] and b[j - 1] == '!':
                    b = b[:j - 1] + b[j + 1:]
                    a = a[:i] + a[i + 1:]
                    change = 'yes'
                    break
                    
                elif b[j] == a[i] and b[j-1] != '!': ### checks to see if there are duplicates, if so just get rid of the one in b
                    b = b[:j] + b[j + 1:]
                    change = 'yes'
                    break
        
        else:
            
            for j in range(len(b)): ### if not negated then check to see if there are negated items later in list, everything is essentially the same
                if b[j] == a[i] and b[j - 1] != '!':
                    b = b[:j] + b[j + 1:]
                    a = a[:i - 1] + a[i + 1:]
                    change = 'yes'
                    break
                    
                elif b[j] == a[i] and b[j - 1] == '!':
                    b = b[:j - 1] + b[j + 1:]
                    change = 'yes'
                    break
        if change == 'yes':
            break
            
    if change == 'no': ### ends recursion if nothing changed 
        ret_list = a + b
        return ret_list
    
    
    return simplify(a, b)

In [23]:
def remove_duplicates(a): ### removes duplicates 
        
    for i in range(len(a)):
        
        if a[i] == '!' or a[i] == 'replacement':
            pass
        else:
            if a[i - 1] == '!':
                for j in range(i + 1, len(a)): ### checks to see if there are items that cancel 
                    if a[j] == a[i] and a[j - 1] != '!':

                        a.pop(i - 1)
                        a.insert(i-1, 'replacement')
                        a.pop(i)
                        a.insert(i, 'replacement')
                        a.pop(j)
                        a.insert(j, 'replacement')

                    elif a[j] == a[i] and a[j - 1] == '!': ### checks to see if there are duplicates to reduce to 1

                        a.pop(j - 1)
                        a.insert(j - 1, 'replacement')
                        a.pop(j)
                        a.insert(j, 'replacement')
            else:
                for j in range(i + 1, len(a)):
                    if a[j] == a[i] and a[j - 1] == '!': ### also checks to see if there are items to cancel 

                        a.pop(i)
                        a.insert(i, 'replacement')
                        a.pop(j - 1)
                        a.insert(j - 1, 'replacement')
                        a.pop(j)
                        a.insert(j, 'replacement')

                    elif a[j] == a[i] and a[j - 1] != '!': ### checks if there are items to simplify

                        a.pop(i)
                        a.insert(i, 'replacement')
                
                
    while 'replacement' in a:
        a.remove('replacement')
        


In [24]:
def prop_resolution(kb, a):
    
    kb = convert_CNF(kb)
    
    clauses = []
    
    clauses += find_clauses(kb) ### finds clauses 
    #print(clauses, '\n')
        
    clauses = clause_final(clauses)
    #print(clauses, '\n')
    
     
    a = '( ! ' + a + ' )'
    
    a = convert_CNF(a) ### converts alpha to CNF 
    
    a_list = a.split()
    
    clauses.append(a_list) ### adds to clauses list 
             
        
    for clause in clauses: ### removes all that is not an atom, ! will be a separate element in the list
        while '(' in clause:
            clause.remove('(')
        while ')' in clause:
            clause.remove(')')
        while '^' in clause:
            clause.remove('^')
        while 'v' in clause:
            clause.remove('v')
        remove_duplicates(clause)
        
    clauses.sort(key = len, reverse = True)
    
    new = []
    
    stop = False
    while stop == False: 
        for i in range(len(clauses)):

            resolvents = []

            for j in range(0, i): ### all cases before this certain clause 
                
                sub = simplify(clauses[i].copy(), clauses[j].copy()) ### simplifies the the 2 clauses 
                #print('1: ', sub)
                if sub == []: ### if item to be added is the empty set then return True
                    return True

                if len(clauses[i]) == len(clauses[j]):

                    if (not (len(sub) > len(clauses[i]))) and sub not in resolvents:
                        resolvents.append(sub)
                else:
                    this = max(len(clauses[i]), len(clauses[j]))

                    high_list = None
                    if this == len(clauses[i]):
                        high_list = clauses[i]
                    elif this == len(clauses[j]):
                        high_list = clauses[j]

                    #if (not (len(sub) > this)) and sub not in resolvents:
                    
                    #doesn't add item if the item is longer than the longest item 
                    if (not (len(sub) - sub.count('!') >  this - high_list.count('!'))) and sub not in resolvents:

                        resolvents.append(sub)


            for j in range(i + 1, len(clauses)): ### all cases after this certain clause 

                sub = simplify(clauses[i].copy(), clauses[j].copy())

                if sub == []:
                    
                    return True  

                if len(clauses[i]) == len(clauses[j]):

                    if (not (len(sub) > len(clauses[i]))) and sub not in resolvents:
                        resolvents.append(sub)
                else:       
                    this = max(len(clauses[i]), len(clauses[j]))

                    high_list = None
                    if this == len(clauses[i]):
                        high_list = clauses[i]
                    elif this == len(clauses[j]):
                        high_list = clauses[j]

                    if (not (len(sub) - sub.count('!') >  this - high_list.count('!'))) and sub not in resolvents:
                        resolvents.append(sub)

            for resolvent in resolvents:
                if resolvent not in new:
                    new.append(resolvent)

            #print('\n')
            #print('new: ', new, '\n')

            if all(x in clauses for x in new): ## if new is a subset of clauses return False 
                return False

            for clause in new: ### adds the new things to the clauses 
                if clause not in clauses:
                    clauses.append(clause)

In [25]:
#rop_resolution('( P ^ ( P => Q ) )', '( ! Q )') # True 

#p11 = A
#B11 = B
#p12 = C
#P21 = D
#B21 = E
#P22 = F
#P31 = G
#wumpus = '( ( ( ! A ) ^ ( B <=> ( C v D ) ) ) ^ ( ( E <=> ( ( A v F ) v G ) ) ^ ( ( ! B ) ^ E ) ) )'

#wumpus = '( ( ! A ) ^ ( B <=> ( C v D ) ) ^ ( E <=> ( A v F v G ) ) ^ ( ! B ) ^ ( E ) )'

#prop_resolution(wumpus, '( ! C )') # True True


#y = mythical
#i = immortal
#o = mortal
#a = mammal
#h = horned
#g = magical

#unicorn = '( ( y => i ) ^ ( ( ! y ) => ( ! i ^ a ) ) ^ ( ( i v a ) => h ) ^ ( h => g ) )'

#prop_resolution(unicorn, '( y )') # False False
#prop_resolution(unicorn, '( g )') # True False
#prop_resolution(unicorn, '( h )') # True False



#knights = '( ' + a + ' ^ ' + b + ' ^ ' + c + ' ^ ' + d + ' ^ ' + e + ' ^ ' + f + ' ^ ' + g + ' ^ ' + h + ' )'
#prop_resolution(knights, '( ! X )') # True True
#prop_resolution(knights, '( ! Y )') # True True 
#prop_resolution(knights, '( Z )') # True True 

In [33]:
a = '( A <=> X )'
b = '( B <=> ( Y v Z ) )'
c = '( C <=> ( A ^ B ) )'
d = '( D <=> ( X ^ Y ) )'
e = '( E <=> ( X ^ Z ) )'
f = '( F <=> ( D v E ) )'
g = '( G <=> ( C => F ) )'
h = '( H <=> ( ( G ^ H ) => A ) )'

def concluder():
    print('Modus Ponens: ')
    print('Knowledge Base: \n', '( P ^ ( P => Q ) )')
    print('Query: Q')
    print(' Ans with model checking: ', entails('( P ^ ( P => Q ) )', '( Q )'))
    print(' Ans with propositional resolution: ', prop_resolution('( P ^ ( P => Q ) )', '( Q )'))
    print('\n\n')
    
    
    print('Wumpus: ')
    wumpus = '( ( ! A ) ^ ( B <=> ( C v D ) ) ^ ( E <=> ( A v F v G ) ) ^ ( ! B ) ^ ( E ) )'
    print('Knowledge Base: \n', wumpus)
    print('Query: ! P12')
    
    wumpus = '( ( ( ! A ) ^ ( B <=> ( C v D ) ) ) ^ ( ( E <=> ( ( A v F ) v G ) ) ^ ( ( ! B ) ^ E ) ) )'
    print(' Ans with model checking: ', entails(wumpus, '( ! C )'))
    
    wumpus = '( ( ! A ) ^ ( B <=> ( C v D ) ) ^ ( E <=> ( A v F v G ) ) ^ ( ! B ) ^ ( E ) )'
    print(' Ans with propositional resolution: ', prop_resolution(wumpus, '( ! C )'))
    print('\n\n')
    
    
    print('Horn Clauses: ')
    unicorn = '( ( y => i ) ^ ( ( ! y ) => ( ! i ^ a ) ) ^ ( ( i v a ) => h ) ^ ( h => g ) )'
    print('Knowledge Base: \n', unicorn)
    print('Query: mythical')
    
    unicorn = '( ( ( y => i ) ^ ( ( ! y ) => ( ( ! i ) ^ a ) ) ) ^ ( ( ( i v a ) => h ) ^ ( h => g ) ) )'
    print(' Ans with model checking: ', entails(unicorn, '( y )'))
    
    unicorn = '( ( y => i ) ^ ( ( ! y ) => ( ! i ^ a ) ) ^ ( ( i v a ) => h ) ^ ( h => g ) )'
    #unicorn = '( ( ( y => i ) ^ ( ( ! y ) => ( ( ! i ) ^ a ) ) ) ^ ( ( ( i v a ) => h ) ^ ( h => g ) ) )'

    print(' Ans with propositional resolution: ', prop_resolution(unicorn, '( y )'))
    print('Query: magical')
    
    unicorn = '( ( ( y => i ) ^ ( ( ! y ) => ( ( ! i ) ^ a ) ) ) ^ ( ( ( i v a ) => h ) ^ ( h => g ) ) )'
    print(' Ans with model checking: ', entails(unicorn, '( g )'))
    
    unicorn = '( ( y => i ) ^ ( ( ! y ) => ( ! i ^ a ) ) ^ ( ( i v a ) => h ) ^ ( h => g ) )'
    print(' Ans with propositional resolution: ', prop_resolution(unicorn, '( g )'))
    print('Query: horned')
    
    unicorn = '( ( ( y => i ) ^ ( ( ! y ) => ( ( ! i ) ^ a ) ) ) ^ ( ( ( i v a ) => h ) ^ ( h => g ) ) )'
    print(' Ans with model checking: ', entails(unicorn, '( h )'))
    unicorn = '( ( y => i ) ^ ( ( ! y ) => ( ! i ^ a ) ) ^ ( ( i v a ) => h ) ^ ( h => g ) )'
    print(' Ans with propositional resolution: ', prop_resolution(unicorn, '( h )'))
    print('\n\n')
    
    
    print('Doors')
    knights = '( ' + a + ' ^ ' + b + ' ^ ' + c + ' ^ ' + d + ' ^ ' + e + ' ^ ' + f + ' ^ ' + g + ' ^ ' + h + ' )'

    print('Knowledge Base: \n', knights)
    print('Query: Door X')
    
    knights = '( ( ( ( ( ( ( ' + a + ' ^ ' + b + ' ) ' + '^ ' + c + ' ) ' + '^ ' + d + ' ) ' + '^ ' + e + ' ) ' + '^ ' + f + ' ) ' + '^ ' + g + ' ) ' + '^ ' + h + ' )'
    print(' Ans with model checking: ', entails(knights, '( X )'))
    
    knights = '( ' + a + ' ^ ' + b + ' ^ ' + c + ' ^ ' + d + ' ^ ' + e + ' ^ ' + f + ' ^ ' + g + ' ^ ' + h + ' )'
    print(' Ans with propositional resolution: ', prop_resolution(knights, '( X )'))
    print('Query: Door Y')
    
    knights = '( ( ( ( ( ( ( ' + a + ' ^ ' + b + ' ) ' + '^ ' + c + ' ) ' + '^ ' + d + ' ) ' + '^ ' + e + ' ) ' + '^ ' + f + ' ) ' + '^ ' + g + ' ) ' + '^ ' + h + ' )'
    print(' Ans with model checking: ', entails(knights, '( Y )'))
    
    knights = '( ' + a + ' ^ ' + b + ' ^ ' + c + ' ^ ' + d + ' ^ ' + e + ' ^ ' + f + ' ^ ' + g + ' ^ ' + h + ' )'
    print(' Ans with propositional resolution: ', prop_resolution(knights, '( Y )'))
    print('Query: Door Z')
    
    knights = '( ( ( ( ( ( ( ' + a + ' ^ ' + b + ' ) ' + '^ ' + c + ' ) ' + '^ ' + d + ' ) ' + '^ ' + e + ' ) ' + '^ ' + f + ' ) ' + '^ ' + g + ' ) ' + '^ ' + h + ' )'
    print(' Ans with model checking: ', entails(knights, '( Z )'))
    
    knights = '( ' + a + ' ^ ' + b + ' ^ ' + c + ' ^ ' + d + ' ^ ' + e + ' ^ ' + f + ' ^ ' + g + ' ^ ' + h + ' )'
    print(' Ans with propositional resolution: ', prop_resolution(knights, '( Z )'))
    print('\n\n')

import time 

start = time.time()
concluder()
end = time.time()

print(end - start)

Modus Ponens: 
Knowledge Base: 
 ( P ^ ( P => Q ) )
Query: Q
 Ans with model checking:  True
 Ans with propositional resolution:  True



Wumpus: 
Knowledge Base: 
 ( ( ! A ) ^ ( B <=> ( C v D ) ) ^ ( E <=> ( A v F v G ) ) ^ ( ! B ) ^ ( E ) )
Query: ! P12
 Ans with model checking:  True
 Ans with propositional resolution:  True



Horn Clauses: 
Knowledge Base: 
 ( ( y => i ) ^ ( ( ! y ) => ( ! i ^ a ) ) ^ ( ( i v a ) => h ) ^ ( h => g ) )
Query: mythical
 Ans with model checking:  True
 Ans with propositional resolution:  False
Query: magical
 Ans with model checking:  True
 Ans with propositional resolution:  True
Query: horned
 Ans with model checking:  True
 Ans with propositional resolution:  True



Doors
Knowledge Base: 
 ( ( A <=> X ) ^ ( B <=> ( Y v Z ) ) ^ ( C <=> ( A ^ B ) ) ^ ( D <=> ( X ^ Y ) ) ^ ( E <=> ( X ^ Z ) ) ^ ( F <=> ( D v E ) ) ^ ( G <=> ( C => F ) ) ^ ( H <=> ( ( G ^ H ) => A ) ) )
Query: Door X
 Ans with model checking:  True
 Ans with propositional resolution:

In [30]:
wumpus = '( ( ! A ) ^ ( B <=> ( C v D ) ) ^ ( E <=> ( A v F v G ) ) ^ ( ! B ) ^ ( E ) )'

prop_resolution(wumpus, '( ! C )') # True True

True

In [27]:
### Added this to help find all the clauses but I don't think it is necessary after testing

def finalize(kb):
    
    kb_list = kb.split()
    
    left = []
    right = []
    
    left_mark = None
    right_mark = None
    
    new_thing = []
    
    count = 0
    for i in range(len(kb_list)):
        if kb_list[i] == '(':
            count += 1
            
        elif kb_list[i] == ')':
            count -= 1

        elif count == 1 and kb_list[i] == 'v':   
            for j in range(i - 1, 0, -1):
                if kb_list[j - 1] == '^' or j == 0:
                    print(j)
                    left = kb_list[j:i + 1]
                    left_mark = j
                    break
                
            for j in range(i + 2, len(kb_list)):
                if kb_list[j] == '^' or j == len(kb_list) - 1:
                    right = kb_list[i + 1: j]
                    right_mark = j
                    break
            
            break
            
    if count == 0 or left == [] or right == []:
        return kb 
    
    left.insert(0, '(')
    right.append(')')
    
    new_thing += kb_list[:left_mark]
    new_thing += left
    new_thing += right
    new_thing += kb_list[right_mark:]
        
    s = ''
    for i in new_thing:
        s += i + ' '
        
    return s